In [17]:
import numpy as np
import pandas as pd
import ta
from vnstock import stock_historical_data
import ta

In [18]:
import vnstock as vn

In [19]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_FAST_PERIOD = 12
MACD_SLOW_PERIOD = 26
MACD_SIGNAL_PERIOD = 9

In [20]:
def calculate_indicators_macd(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)
    
    return df

In [21]:
def trading_strategy_test_macd(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_MACD'] < df['Previous_Signal_Line']) & (df['MACD'] >= df['Signal_Line']) & (df['RSI'] > RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [22]:
start_date = '2017-07-24'
end_date = '2019-02-18'

In [23]:
non_ESG =  ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']

In [24]:
VN30 = ['CII','CTD','CTG','DHG','DPM','EIB','FPT',
        'GAS','GMD','HDB','HPG','MBB','MSN','MWG','NVL',
        'PNJ','REE','ROS','SAB','SBT','SSI','STB','TCB',
        'VCB','VHM','VIC','VJC','VNM','VPB','VRE']

In [25]:
def company_winrate(symbol, start_date, end_date, resolution='1D', type='stock'):
    df = vn.stock_historical_data(symbol=symbol, 
                            start_date=start_date, 
                            end_date=end_date, resolution=resolution, type=type)
    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_macd(df)
    winrate = trading_strategy_test_macd(df)
        
    return winrate

In [26]:
winrate_macd = {}
for company in VN30:
    try:
        company_winr = company_winrate(company, start_date=start_date, end_date=end_date, resolution='1D', type='stock')
        winrate_macd[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

In [27]:
winrate_macd

{'CII': {5: 41.66666666666667,
  10: 63.63636363636363,
  15: 63.63636363636363,
  20: 45.45454545454545,
  25: 27.27272727272727,
  30: 36.36363636363637},
 'CTD': {5: 33.33333333333333,
  10: 41.66666666666667,
  15: 33.33333333333333,
  20: 50.0,
  25: 50.0,
  30: 50.0},
 'CTG': {5: 57.14285714285714,
  10: 64.28571428571429,
  15: 64.28571428571429,
  20: 64.28571428571429,
  25: 61.53846153846154,
  30: 61.53846153846154},
 'DHG': {5: 64.28571428571429,
  10: 42.857142857142854,
  15: 42.857142857142854,
  20: 50.0,
  25: 38.46153846153847,
  30: 15.384615384615385},
 'DPM': {5: 45.45454545454545,
  10: 36.36363636363637,
  15: 63.63636363636363,
  20: 50.0,
  25: 60.0,
  30: 70.0},
 'EIB': {5: 28.57142857142857,
  10: 64.28571428571429,
  15: 50.0,
  20: 57.14285714285714,
  25: 69.23076923076923,
  30: 61.53846153846154},
 'FPT': {5: 73.33333333333333,
  10: 60.0,
  15: 46.666666666666664,
  20: 60.0,
  25: 57.14285714285714,
  30: 57.14285714285714},
 'GAS': {5: 57.142857142857

In [29]:
winrate_macd = pd.DataFrame(winrate_macd)

In [33]:
winrate_macd.to_excel('winrate_result/macd.xlsx')

In [34]:
def calculate_indicators_bb(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    return df

In [35]:
def trading_strategy_test_bb(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['close'] <= df['Bollinger_low']) & (df['RSI'] < RSI_OVERSOLD)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [36]:
def company_winrate_bb(symbol, start_date, end_date, resolution='1D', type='stock'):
    df = vn.stock_historical_data(symbol=symbol, 
                            start_date=start_date, 
                            end_date=end_date, resolution=resolution, type=type)
    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_bb(df)
    winrate = trading_strategy_test_bb(df)
        
    return winrate

In [37]:
winrate_bb = {}
for company in VN30:
    try:
        company_winr = company_winrate_bb(company, start_date=start_date, end_date=end_date, resolution='1D', type='stock')
        winrate_bb[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

In [40]:
winrate_bb = pd.DataFrame(winrate_bb)

In [41]:
winrate_bb.to_excel('winrate_result/bb.xlsx')

In [42]:
OBV_PERIOD = 5

In [43]:
def calculate_indicators_obv(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

In [44]:
def trading_strategy_test_obv(df, day_range=(5, 30), step=5):
    results = {}
    
    for days in range(day_range[0], day_range[1] + 1, step):
        df['Buy_Signal'] = (df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0)
        wins = 0
        total_trades = 0
        for i in range(len(df)):
            if df['Buy_Signal'].iloc[i]:
                buy_price = df['close'].iloc[i]
                if i + days < len(df):
                    sell_price = df['close'].iloc[i + days]
                    if sell_price > buy_price:
                        wins += 1
                    total_trades += 1

        win_rate = (wins / total_trades * 100) if total_trades != 0 else 0
        results[days] = win_rate
    
    return results

In [45]:
def company_winrate_obv(symbol, start_date, end_date, resolution='1D', type='stock'):
    df = vn.stock_historical_data(symbol=symbol, 
                            start_date=start_date, 
                            end_date=end_date, resolution=resolution, type=type)
    if df.empty:
        return [0, 0] 
    
    df = calculate_indicators_obv(df)
    winrate = trading_strategy_test_obv(df)
        
    return winrate

In [46]:
winrate_obv = {}
for company in VN30:
    try:
        company_winr = company_winrate_obv(company, start_date=start_date, end_date=end_date, resolution='1D', type='stock')
        winrate_obv[company] = company_winr

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

Error in API response {"status":400,"code":"BAD_REQUEST","message":"invalid symbol"}
 

Error occurred for ROS: cannot access local variable 'df' where it is not associated with a value


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


A valu

In [47]:
winrate_obv = pd.DataFrame(winrate_obv)
winrate_obv.to_excel('winrate_result/obv.xlsx')